# 1. File description 

To feed a model for it to predict a bot from a legit users I need the data to be labelled meaning each user in the dataset needs to have a 1 or 0 for bot/not bot. This file is meant to do so.

To label the data, multiple techniques are gonna be used and are divided in three. Manual, semi-manual and automatic labelling. 

Automatic: 
* Look at bot labelled and check for other users with the same photo. From previous iteration of the project, I have a folder with about 800 of bots photos that I previously found manually. Now, I just check if a user's profile pic is similar to any, if yes, it's a bot. That's the easiest way and the only one where I don't have to check other details of the bot as if two users have the same photo and one is a bot, the other is too.

Semi-manual:
* Look at the bots having a scam domain.
* Use NSFW pre-trained models to get images score for Neutral, Drawing, Hentai, Sexual, Porn and looking at the profile that have a high score for porn + sexy as most bots have nude photos
* Look at a subset of data where I belive bots are
    * Around 200 likes on their comment
    * 12 or less photos
    * Always have a link in their bio
* Find same/similar comments from labelled users 
* Pattern in Bio/description
* Look at the keyword of the flagged bots in comments and profile, look at profile that are similar to bots. 
* After labelling abunch of users, build a weak model to help in the labelling by making predictions. As long as the model is better than random, it'll be helpful.



## 0. Import module and setting notebook preferences

### 0.1 Import modules

In [52]:
import src.useful as su

import pandas as pd 
import numpy as np
import sqlite3 

import plotly.io as pio

# For image manipulation 
from PIL import Image, ImageDraw, ImageFont, UnidentifiedImageError

import os
import shutil

### 0.2 Notebook preferences

In [35]:
pd.set_option("display.max_columns", None)
pio.templates.default = "plotly_dark"

# Creating SQL database to store all the data for the project
database = "data/main_database.sqlite"
con = sqlite3.connect(database)

## 1. Manual labelling 

To label the data manually, the process has to be fast, I need to see all the information about the users condensed in one summary. For that, I chose to create for each user an image with their attributes and their photos. I'll then create a script to open one image by one, and then chose bot/not bot/maybe bot with keyboard arrows. 



In [36]:
# Load the data and display only the one from a 
query = '''
select 
    * 
from comments
'''

df_comments = pd.read_sql_query(query, con)
pages = list(set(df_comments['page']))

### 1.1 UDFs

In [92]:
def generate_card(username):
    df_user = df_main[df_main["username"]==username].reset_index(drop=True)

    # Create new image
    global_image = Image.new("RGB", (1300, 550), "white")
    draw = ImageDraw.Draw(global_image)
    font = ImageFont.truetype("/Users/marclamy/Downloads/Open_Sans/static/OpenSans/OpenSans-Light.ttf", 15)

    # Open user pp and add it to the global image
    try:
        path = f"data/photos/user_profile_pictures/{username}_pp_user_photo.png"
        profile_pic = Image.open(path)
        global_image.paste(profile_pic, (250, 0))
    except FileNotFoundError: 
        pass

    # Creating a list of position to add the photos on the global image
    positions = []
    for x in range(850, 1300, 150):
        for y in range(0, 450, 150):
            positions.append((x, y))


    # Loop trhough each image and add it to the global image
    # user_last12_path = f"data/7_user_last_12_csv/{username}_last12.csv"
    # if os.path.exists(user_last12_path):
    #     last_12 = pd.read_csv(user_last12_path)
    for image_num in range(12):
        path = f"data/photos/user_last_12_posts/{username}_{str(image_num)}_user_photo.png"
        
        try:
            if os.path.exists(path):
                img = Image.open(path)
                if image_num < 9:
                    global_image.paste(img.resize((150, 150)), positions[image_num])
                elif image_num == 9:
                    global_image.paste(img.resize((150, 150)), (400, 0))
                elif image_num == 10:   
                    global_image.paste(img.resize((150, 150)), (550, 0))
                elif image_num == 11:
                    global_image.paste(img.resize((150, 150)), (700, 0))
        except (OSError, UnidentifiedImageError) as e: # For truncated image file
            print("OSError/PIL: ", e)



    # Add basic information to the global image
    username = df_user.loc[0, "username"]
    draw.text((10, 5), f"Username: {username}", font=font, fill="black")

    follower_count = df_user.loc[0, "follower_count"]
    color = "black" if follower_count > 800 else "red"
    draw.text((10, 25), f"Followers: {follower_count:,}", font=font, fill=color)

    following_count = df_user.loc[0, "follow_count"]
    draw.text((10, 45), f"Following: {following_count:,}", font=font, fill="black")

    post_count = df_user.loc[0, "post_count"]
    color = "black" if post_count > 16 else "red"
    draw.text((10, 65), f"Post count: {post_count}", font=font, fill=color)

    video_count = df_user.loc[0, "video_count"]
    draw.text((10, 85), f"Video count: {video_count}", font=font, fill="black")

    comment_likes = round(df_user["comment_likes"].mean())
    draw.text((10, 105), f"Comment_likes: {comment_likes}", font=font, fill="black")

    external_url = df_user.loc[0, "domain"]
    draw.text((10, 125), f"{external_url}", font=font, fill="black")

    biography = df_user.loc[0, "biography"]
    draw.text((10, 145), f"{biography}", font=font, fill="black")


    # Save screenshot, open it, resize it, add it to global_image and deleting the file
    path = f"igbotdata/website_screenshot/{username}_website_photo.png"
    try:
        screenshot = Image.open(path)
        global_image.paste(screenshot.resize((500, 300)), (350, 150))
    except (UnidentifiedImageError, FileNotFoundError):
        pass


    # Export image
    # global_image.save(f"data/igbot_image_summary/move_summary{username}_image_summary.png")
    return global_image





username = np.random.choice(df_main.query('post_count >= 12')['username'])
generate_card(username)